In [10]:
 # Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

In [4]:
file = pd.read_csv("../output_data/city.csv")
file.head()

,City ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,Sorong,40,ID,1650909014,84,-0.8833,131.2500,79.48,1.10
1,1,Sioux Lookout,100,CA,1650909015,73,50.1001,-91.9170,17.67,12.66
2,2,Rikitea,100,PF,1650908901,78,-23.1203,-134.9692,75.58,13.29
3,3,Vaini,100,TO,1650908675,94,-21.2000,-175.2000,71.76,18.41
4,4,Kruisfontein,55,ZA,1650908676,81,-34.0033,24.7314,57.88,6.35


In [5]:
# Configure gmaps.
# Use the Lat and Lng as locations and Humidity as the weight.
# Add Heatmap layer to map.

# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [6]:
# Store 'Lat' and 'Lng' into  locations
locations = file[["Lat", "Lng"]]
humidity = file["Humidity"].astype(float)

In [7]:
# Plot Heatmap
fig = gmaps.figure(center = [0,0] ,zoom_level = 2)

#Create and add heat layer 
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                               dissipating=False, max_intensity=100,
                               point_radius = 4)
fig.add_layer(heat_layer)
#Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [8]:
# Narrow down the cities to fit weather conditions.
# Drop any rows will null values.

temp_file=file.loc[(file['Max Temp']>=70)&(file['Max Temp']<=80)]
wind_file=temp_file.loc[(temp_file['Wind Speed']<=10)]
ideal_weather=wind_file.loc[(wind_file['Cloudiness']==0)]
ideal_weather.head()

,City ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
50,50,Castro,0,BR,1650908708,48,-24.7911,-50.0119,77.54,3.42
83,83,Lompoc,0,US,1650908729,64,34.6391,-120.4579,79.70,4.61
148,148,Half Moon Bay,0,US,1650908767,64,37.4636,-122.4286,71.46,5.75
156,156,Pisco,0,PE,1650908773,83,-13.7000,-76.2167,70.02,8.05
280,280,Illapel,0,CL,1650908856,39,-31.6308,-71.1653,75.09,7.67


In [11]:
hotel_df = ideal_weather.reset_index(drop=True)
hotel_df["Hotel Name"] = ""

# params dictionary to update each iteration
params = {
    "radius": 50000,
    "types": "hotel",
    "keyword": "hotels",
    "key": g_key
}

# Use the lat/lng we recovered to identify hotels
for index, row in hotel_df.iterrows():
    
    # get lat, lng 
    lat = row["Lat"]
    lng = row["Lng"]
    
    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    # Use the search term: "Hotels" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # make request and print url
    hotel_name = requests.get(base_url, params=params)
    
    # convert to json
    hotel_name = hotel_name.json()
    print(json.dumps(hotel_name, indent=4, sort_keys=True))
    
    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_name["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
hotel_df

{
    "html_attributions": [],
    "next_page_token": "Aap_uECTI-MOVq0rzSiI0DBMe8sWN8iU_-utbbUlEUxMBx5SAk_Jbxk39rdtTqA5o03ZWWzNGEegfftquOdK_rvtgc4Wbi80YgtNIo0lqMH_196CoOgTH8IsCCAZB8zN_zE7sXKtaa3zoTFOoN04dzDSAoKSQCX49pwagMgdHLSZ4OefvaDZ2GGEQGIvT0ew0rWK5xYxkXouCSOVlgYXTYKryeSdWwgVJkRDP81dmb0NVAOlDhqyGX-hjQ9aq4NR_SFrU-EbkTXH43qLarCtJ_ZCkj3QKTny6XgrXmFuqms1W-Ga5sfkH2XRejdnJxd8p37YuUxVl7_s14OgXbT4fRXEdyKUGPR4BvIPvHgVBAXr8_ZEx9yvASuDWLz9y4ZmlsZkS7THl_OJlt6-OfBbHq0s8CL2fJ0QpCVqNPZic6fPuLA5F-a12fE6vRCVJYwsofSH",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -25.0995624,
                    "lng": -50.1622834
                },
                "viewport": {
                    "northeast": {
                        "lat": -25.09820397010727,
                        "lng": -50.16084502010728
                    },
                    "southwest": {
                        "lat": -25.1009

{
    "html_attributions": [],
    "next_page_token": "Aap_uEBzky9FPrp9zKg3jBxxwKwoXQFcMFTfXFKHWunl0-vdfcMZ4SXPigKVCEtZ1UmVyvb6yxOnuciu1gAG2AG_Tw6saHLjzf1zJRIeXcIj_l-S8wX3icJt0tjF2oKLapNlepR3ttNgVla7-v6RjVHwn-KEq_vB-JzrzORRmieXY2ejjF4IG1Vc4aVYgTAQ7mseZxYZFll_pqOEx8DyUEZioJNF5zmLMJ4Rh1FKWN1ye7oLieBMF0fxcK-l8zLfhIu2wJYLLk5mC0-HWRWWjyELboAiAnd6tBKoGSD391_5Higv2IR3iQ-LLFZJHlz-piQjkckBcxV0XRhb-m4yBDG-4G3oV02QwDmLIYdE9AVChxXm-T2_hjFtwi3eLKcvfrFovjRTR87NKRAGI9iv4uozdihG48eJPGsWt1VsbSzEepmU48hbHsS8QnNPF2w6j7Yx",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 34.609066,
                    "lng": -120.0862503
                },
                "viewport": {
                    "northeast": {
                        "lat": 34.61054797989272,
                        "lng": -120.0853049701073
                    },
                    "southwest": {
                        "lat": 34.6078483

{
    "html_attributions": [],
    "next_page_token": "Aap_uEB633n1GUQG8WleAXzo51QTLSn_rPevg3LpOoZiTpgqv3O7R54cqnxc-_r_CWDrXXubQnrj98aCBN4hyKGmARlqFGzpMNHyrXsptLNjM-aA2T1MvuuQefgVTJ0tZTKX2xjxjs9KDyb81tL4CgivlzegIfSIjf3rtzFW4yM6mONJ1XVOTQm9FlJEdaO6Sc6nuzs0B801mhEc_HMvWXbFphN8nBUFjWNt_AJ-Rn217qpK36sGZx9lhbSBDhtPs0wCiorAzofyMC9ybCeQZvKakqXJKadFQxxhkgqHDKNecKQNPmwLW3iP7iKgdaq5qnk4HcvIhsYNqKaP5gY3D6zCkh5aNuahvesqKF6WQL7uHtTWZ0JptKYkzwSEk2TppL4gxnrb5-04BxEewk90hHOLZYNuuCL2ti1ZHuP_R8zhfhwHkb8o1gJw4X2tGgBiyaVO",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 37.8729676,
                    "lng": -122.4562106
                },
                "viewport": {
                    "northeast": {
                        "lat": 37.87446162989271,
                        "lng": -122.4549371701073
                    },
                    "southwest": {
                        "lat": 37.871761

{
    "html_attributions": [],
    "next_page_token": "Aap_uECl9tRjxFDPDqlvxxqF-i7coRyUHMVYeVkGfinZJar2GRHFeAwLoQVqq0IP19RznSPqkxM8KlhkiUQO5zxpVuu_4nbvjCu9pYjE1lgiu-QYlRKBjWSH4BP8zIL55mLxfPQ8z-A4bIXnanuChwlRfSZLKdBAusU69ILDb_l6aUGuqxyPm68QYpwAcr2h_wsOI7yfkFEt-PeUfsPaWRqbrEAxT2xYrYM1OP0_hdWz0eHuuQeSd74NTytK-kSZ0LI7RC_gLZWuDpOXfdAMFlfv48ZZzDUw-kNTG1BZBmE6oq0SMwYNe85XdskwiJ_g0BB6wTchgPKFk29Q846qBuk0418v1uP8An7CQ41zBTRQXJY2YC0_GYuP4NIiR2ldGJCv61R2PtI_U7AAK9q5A_PUOxdsxr2QJ68rHWXsnmHZGXJylvo9lNhvj-FWHjfipdeE",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -14.0477062,
                    "lng": -75.7528004
                },
                "viewport": {
                    "northeast": {
                        "lat": -14.04651637010728,
                        "lng": -75.75124737010727
                    },
                    "southwest": {
                        "lat": -14.0492

{
    "html_attributions": [],
    "next_page_token": "Aap_uEAvPCehsssnpC7gReIxcWwGL4XBytkVPPpUVyJ0bwiYcJdChjOxzRWzyZXgAH55C8Y3KT4opxd3q94EpFRfs2vDqzXmdO9QSL-qT5EUg076Jl_caDoA1LzUDVEsq9Dv00mCEtXKmGPpr-vE1lqDNasvewNezDuY1JqI08b1O0qb6Qculp3vnm_awGqXZP_rrh4DV_jIxOXp-tQmDxgetshB4wJkxEor4Ai7M5LI18UIabOSGUId4_0NnDqhgo4Z7_LeY0PyxolXua489LU9mVPfXCZteTZ2j4b2KA8venULMHyTP8PEnhwAI7PFk_qWwzsXMxWwfYeTJhWWp5FeVbXzpSC63zyKKMIGRzwElE6SRtMTae99PxCaSbEIo84cEL8s4A9qCVyMZ48LBPRgwr61_0RhTSRtKBvWAduozOBRovv2iuVGHJMFXym0979z",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -31.8832497,
                    "lng": -70.7300841
                },
                "viewport": {
                    "northeast": {
                        "lat": -31.88200392010727,
                        "lng": -70.72875007010728
                    },
                    "southwest": {
                        "lat": -31.8847

...

{
    "html_attributions": [],
    "next_page_token": "Aap_uEBBjxupIG8A1yU0ngNfo3lI2sGIrVNUmFYYlCwoityjPw7DoL84hJ7UapbqXmb3Sm3a45gRs27jjXzxuDywaXJCytyyzU-8iTd2uSHZDlkIZyzElZqgXAE9M_UEJOvJZqCEbKtiuZRbbNrSikgjx1koWsrVAWiJzWOfk9fVa23RhMqZh8xhPoTaOcPG50UP7a8zZ-YWI1tXgvDly9Q6MZ_KgAck6IzPknHlh9wUqs25dbU4qvk9_JogngjHWr3YrAxM_D034ALL8dMjHeFNK5izg8e35A2I2q7B1f6t03z8AYadqoGRGp0GoFUCvexkcHk6dokBVZlSBYHNGyCQvM2YiJN7whiEE05fZs0CQsbclXVKjdeKV2ceKgYztbAmgNslLyubvcVik33ZNtX4jnpkACpj__FLeFcS0q7xL2AnCI6oAvkk31F3-HwEaDDw",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 32.7923826,
                    "lng": 35.5413119
                },
                "viewport": {
                    "northeast": {
                        "lat": 32.79376197989271,
                        "lng": 35.54277522989272
                    },
                    "southwest": {
                        "lat": 32.791062320

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 30.4003812,
                    "lng": -115.9119201
                },
                "viewport": {
                    "northeast": {
                        "lat": 30.40211027989272,
                        "lng": -115.9107055701073
                    },
                    "southwest": {
                        "lat": 30.39941062010728,
                        "lng": -115.9134052298927
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "icon_background_color": "#909CE1",
            "icon_mask_base_uri": "https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet",
            "name": "Mision Santa Maria",
            "photos": [
                {
                    "height": 4

{
    "html_attributions": [],
    "next_page_token": "Aap_uEAag0temwncnvWZ7tfGUJeZLBBIv14vlvV1tW_NuovjiB_8EoZusObQYMkC6WAKTp_qCMUq9B4BlTr9ni0Xfh8vCczDa3ZbtCIVBiiUXGHk2ofot0pap-AxvTJPblJTvYLsaRtiIvYpkEXGIma6Lt0evcjAuuGAyHDDxwnAMF0rauNzZJnH1XYGXjSf9De3wc6rhIfxPtzVgPnnWWHwSEwI_rH573oVV93WLLczqMFOvyIY0LdAjuWJOWNzG1BZdrQpBSEaXo-eI8p5HbLRDzOjqVaDr7luroPQuS_Ymt1SHmcRFqPKn2fR_iwokX8JJtIeTml1LqC7mNc21h3RHoW0jw8pHYuqbESxyZWmJfSfjOKMMGioIUeX1LHZAnTFhOyHnWcWtMRopgbM-BuKFuWLcIxfvXs3v_nKnrAFoSOpiYcSAg3OH1sOTGwvS7mR",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 37.7855643,
                    "lng": -122.4085222
                },
                "viewport": {
                    "northeast": {
                        "lat": 37.78684092989273,
                        "lng": -122.4071512201073
                    },
                    "southwest": {
                        "lat": 37.784141

{
    "html_attributions": [],
    "next_page_token": "Aap_uEANJt37xrvYbzCV1JGdMEneo3Dt9NJRhQBdRTEEW_6N-iDw6HcN-t03dlHzkMg8BcMc1RcsRCaYJoIqjpMwPjkZYnluJ0v-DCvauKJGCY9RPceKDZJ7k9lriCbBAoQxQkMNfOqJqGzwkhP3LIrwNHyfL90REAZFYqGEP4ydYqOZpgbLbuccRBmktCRxRasTQUTaUVvMmJ3Jy6AL4pg9VDAOO3uo_59HRNZXfmlxXkg4MQ2yq6GRfDp8N47Fi0Pq3hvGi_HlbOBLPrZlBcal-JOwX8xs4t4uTV0I5eyXTaCQf_yG_fac_4v5-F22nOo2xrtssKgOxj21FECRu6ZkTqTFnx7dPM2OYOQoKEdNS8HXe1-rlnTLPLOVZFgSs-YYjAG0MuTu53LrBCMz-LJmRYQaTb0Ntg5KAI8mY5Zf1vo0VuIgQSyk8pukQ7vVGgAE",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 38.4686768,
                    "lng": -121.8236894
                },
                "viewport": {
                    "northeast": {
                        "lat": 38.47021542989272,
                        "lng": -121.8221939201073
                    },
                    "southwest": {
                        "lat": 38.467515

,City ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,50,Castro,0,BR,1650908708,48,-24.7911,-50.0119,77.54,3.42,Premium Vila Velha Hotel
1,83,Lompoc,0,US,1650908729,64,34.6391,-120.4579,79.70,4.61,Chumash Casino Resort
2,148,Half Moon Bay,0,US,1650908767,64,37.4636,-122.4286,71.46,5.75,Waters Edge Hotel
3,156,Pisco,0,PE,1650908773,83,-13.7000,-76.2167,70.02,8.05,Hotel Las Dunas
4,280,Illapel,0,CL,1650908856,39,-31.6308,-71.1653,75.09,7.67,Suite Euro
5,380,Vallenar,0,CL,1650908920,26,-28.5708,-70.7581,79.75,7.92,Hotel Origenes
6,429,Nahariya,0,IL,1650908953,88,33.0059,35.0941,71.51,8.77,Leonardo Hotel Tiberias
7,436,San Quintín,0,MX,1650908955,27,30.4833,-115.9500,75.15,7.09,Mision Santa Maria
8,459,San Ramon,0,US,1650908963,55,37.7799,-121.9780,71.82,3.00,Hotel Fusion
9,501,Laguna,0,US,1650908978,40,38.4210,-121.4238,73.02,5.75,"Country Inn & Suites by Radisson, Dixon, CA - ..."


In [12]:
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [13]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations,
    info_box_content=[f"Hotel loc: {rate}" for rate in hotel_info])

fig.add_layer(markers)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))